In [ ]:
# --- Force working directory to this file's folder and build robust paths ---
import os, sys
from pathlib import Path
from configs.test_config import TestConfig
from network_definition import NetworkDefinition
from pypsa2smspp.transformation import Transformation
from datetime import datetime
import pysmspp

from pypsa2smspp.network_correction import (
    clean_marginal_cost,
    clean_global_constraints,
    clean_e_sum,
    clean_efficiency_link,
    clean_ciclicity_storage,
    clean_marginal_cost_intermittent,
    clean_storage_units,
    clean_stores,
    parse_txt_file,
    compare_networks,
    add_slack_unit
    )

def get_datafile(fname):
    return os.path.join(os.path.dirname(__file__), "test_data", fname)

#%% Network definition with PyPSA
config = TestConfig()
nd = NetworkDefinition(config)

nd.n = add_slack_unit(nd.n)
nd.n = clean_ciclicity_storage(nd.n)


network = nd.n.copy()
network.optimize(solver_name='gurobi')

# network.export_to_netcdf("test_pypsa.nc")

network.model.to_file(fn = "pypsa.lp")
#%% Transformation class
then = datetime.now()
transformation = Transformation(network, merge_links=True, expansion_ucblock=True)
print(f"La classe di trasformazione ci mette {datetime.now() - then} secondi")

tran = transformation.convert_to_blocks()


NameError: name '__file__' is not defined

In [ ]:
if transformation.dimensions['InvestmentBlock']['NumAssets'] == 0 or transformation.expansion_ucblock:
    ### UCBlock configuration ###
    configfile = pysmspp.SMSConfig(template="UCBlock/uc_solverconfig_grb")  # load a default config file [highs solver]
    temporary_smspp_file = "output/network_uc_hydro_0011.nc"  # path to temporary SMS++ file
    output_file = "output/temp_log_file.txt"  # path to the output file (optional)
    solution_file = "output/solution_uc_hydro_0011.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, log_executable_call=True)
    
    statistics = network.statistics()
    operational_cost = statistics['Operational Expenditure'].sum()
    # error = (operational_cost - result.objective_value) / operational_cost * 100

    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    
    # Esegui la funzione sul file di testo
    data_dict = parse_txt_file(output_file)

    print(f"Il solver ci ha messo {data_dict['elapsed_time']}s")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    
    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    # transformation.parse_txt_to_unitblocks(output_file)
    transformation.inverse_transformation(nd.n)

    differences = compare_networks(network, nd.n)
    statistics_smspp = nd.n.statistics()
    

else:
    ### InvestmentBlock configuration ###
    configfile = pysmspp.SMSConfig(template="InvestmentBlock/BSPar.txt")
    temporary_smspp_file = "output/temp_network_investment_daily.nc"
    output_file = "output/temp_log_file_investment.txt"  # path to the output file (optional)
    solution_file = "output/temp_solution_file_investment.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, inner_block_name='InvestmentBlock', log_executable_call=True)
    
    
    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    transformation.inverse_transformation(nd.n)

Executing command:
ucblock_solver /home/pampado/pypsa2smspp/test/output/network_uc_hydro_0011.nc -c /home/pampado/pySMSpp/pysmspp/data/configs/UCBlock/ -S uc_solverconfig_grb.txt -o -O /home/pampado/pypsa2smspp/test/output/solution_uc_hydro_0011.nc



Error PyPSA-SMS++ of -0.2797603805589163%
Il solver ci ha messo 0.001864603s
Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è 0:00:00.375848
[split] 'DCNetworkBlock_3' -> 'IT0 0 H2 Electrolysis' + 'IT0 0 H2 Fuel Cell'


In [ ]:
network.statistics()

Optimal Capacity  Installed Capacity       Supply  \
Generator slack                  5441.84481          5441.84481  38470.47553   
          solar                 71058.27560         19711.86918  25856.39421   
Link      H2 electrolysis         100.00000           100.00000      0.00000   
          H2 fuel cell            100.00000           100.00000      0.00000   
Load      AC                        0.00000             0.00000      0.00000   
Store     H2                      365.00000           365.00000    365.00000   

                            Withdrawal  Energy Balance Transmission  \
Generator slack                0.00000     38470.47553          0.0   
          solar                0.00000     25856.39421          0.0   
Link      H2 electrolysis    632.25359      -632.25359          0.0   
          H2 fuel cell       365.00000      -365.00000          0.0   
Load      AC               63877.11614    -63877.11614          0.0   
Store     H2                 365.00000         0.00000          0.0   

                           Capacity Factor   Curtailment  Capital Expenditure  \
Generator slack                   0.294558   92133.79993         0.000000e+00   
          solar                   0.015161  430764.08482         3.908205e+06   
Link      H2 electrolysis         0.263439       0.00000         2.516210e+07   
          H2 fuel cell            0.152083       0.00000         1.323257e+07   
Load      AC                           NaN       0.00000         0.000000e+00   
Store     H2                      0.483503       0.00000         8.120717e+04   

                           Operational Expenditure       Revenue  Market Value  
Generator slack                       3.847048e+08  3.847048e+08  10000.000002  
          solar                       2.585639e+02  3.908464e+06    151.160432  
Link      H2 electrolysis             0.000000e+00 -6.322540e+00           NaN  
          H2 fuel cell                0.000000e+00 -1.825000e+06           NaN  
Load      AC                          0.000000e+00 -3.904382e+08           NaN  
Store     H2                          0.000000e+00  1.824994e+06   4999.983774

In [ ]:
nd.n.statistics()

Optimal Capacity  Installed Capacity       Supply  \
Generator slack                  5441.84481          5441.84481  38470.47553   
          solar                 19711.86918         19711.86918  25856.39421   
Link      H2 electrolysis         100.00000           100.00000      0.00000   
          H2 fuel cell            100.00000           100.00000      0.00000   
Load      AC                        0.00000             0.00000      0.00000   
Store     H2                      365.00000           365.00000    182.50000   

                            Withdrawal  Energy Balance Transmission  \
Generator slack                0.00000     38470.47553          0.0   
          solar                0.00000     25856.39421          0.0   
Link      H2 electrolysis    632.25359      -632.25359          0.0   
          H2 fuel cell       182.50000      -182.50000          0.0   
Load      AC               63877.11614    -63877.11614          0.0   
Store     H2                 632.25359      -449.75359          0.0   

                           Capacity Factor   Curtailment  Capital Expenditure  \
Generator slack                   0.294558   92133.79993         0.000000e+00   
          solar                   0.054655  102711.89421         1.084153e+06   
Link      H2 electrolysis         0.263439       0.00000         2.516210e+07   
          H2 fuel cell            0.076042       0.00000         1.323257e+07   
Load      AC                           NaN       0.00000         0.000000e+00   
Store     H2                      0.335925       0.00000         8.120717e+04   

                           Operational Expenditure  Revenue  Market Value  
Generator slack                       3.847048e+08      0.0           NaN  
          solar                       2.585639e+02      0.0           NaN  
Link      H2 electrolysis             0.000000e+00      0.0           0.0  
          H2 fuel cell                0.000000e+00      0.0           0.0  
Load      AC                          0.000000e+00      0.0           0.0  
Store     H2                          0.000000e+00      0.0           NaN

In [ ]:
# a = transformation.unitblocks['BatteryUnitBlock_3']['variables']['ConverterMaxPower']

In [ ]:
network.objective / 1e8

3.8752906620959564

In [ ]:
import pandas as pd
df1 = nd.n.stores_t['p']      # primo DataFrame
df2 = network.stores_t['p']   # secondo DataFrame
df1.join(df2, how='inner', lsuffix='_nd', rsuffix='_net')

Store,IT0 0 H2_nd,IT0 0 H2_net
snapshot,,
0,0.000000,0.00
1,0.000000,0.00
2,0.000000,0.00
3,0.000000,0.00
4,-100.000000,0.00
5,0.000000,-57.73
6,0.000000,-57.73
7,0.000000,-57.73
8,0.000000,-57.73


In [ ]:
# --- Multi-column case (align on common link names) ---
p0_nd  = nd.n.links_t['p0']
p0_net = network.links_t['p0']
p1_nd  = nd.n.links_t['p1']
p1_net = network.links_t['p1']

common_links_p0 = p0_nd.columns.intersection(p0_net.columns)
common_links_p1 = p1_nd.columns.intersection(p1_net.columns)

p0_cmp = p0_nd[common_links_p0].join(p0_net[common_links_p0], lsuffix='_nd', rsuffix='_net')
p1_cmp = p1_nd[common_links_p1].join(p1_net[common_links_p1], lsuffix='_nd', rsuffix='_net')

# Pack everything together with a neat column MultiIndex: ('p0','battery_nd'), ('p0','battery_net'), ...
links_merged = pd.concat({'p0': p0_cmp, 'p1': p1_cmp}, axis=1)

# Diffs for common links
p0_diff = p0_nd[common_links_p0] - p0_net[common_links_p0]
p1_diff = p1_nd[common_links_p1] - p1_net[common_links_p1]

links_merged

p0                        \
Link     IT0 0 H2 Electrolysis_nd IT0 0 H2 Fuel Cell_nd   
snapshot                                                  
0                        0.000000                   0.0   
1                        0.000000                   0.0   
2                        0.000000                   0.0   
3                        0.000000                   0.0   
4                      100.000000                   0.0   
5                        0.000000                   0.0   
6                        0.000000                   0.0   
7                        0.000000                   0.0   
8                        0.000000                   0.0   
9                       32.253594                   0.0   
10                     100.000000                   0.0   
11                     100.000000                   0.0   
12                     100.000000                   0.0   
13                     100.000000                   0.0   
14                     100.000000                   0.0   
15                       0.000000                   0.0   
16                       0.000000                   0.0   
17                       0.000000                  32.5   
18                       0.000000                  50.0   
19                       0.000000                  50.0   
20                       0.000000                  50.0   
21                       0.000000                   0.0   
22                       0.000000                   0.0   
23                       0.000000                   0.0   

                                                           \
Link     IT0 0 H2 Electrolysis_net IT0 0 H2 Fuel Cell_net   
snapshot                                                    
0                         0.000000                    0.0   
1                         0.000000                    0.0   
2                         0.000000                    0.0   
3                         0.000000                    0.0   
4                         0.000000                    0.0   
5                       100.000000                    0.0   
6                       100.000000                    0.0   
7                       100.000000                    0.0   
8                       100.000000                    0.0   
9                       100.000000                    0.0   
10                       32.253594                    0.0   
11                        0.000000                    0.0   
12                      100.000000                    0.0   
13                        0.000000                    0.0   
14                        0.000000                    0.0   
15                        0.000000                    0.0   
16                        0.000000                    0.0   
17                        0.000000                    0.0   
18                        0.000000                  100.0   
19                        0.000000                   65.0   
20                        0.000000                  100.0   
21                        0.000000                  100.0   
22                        0.000000                    0.0   
23                        0.000000                    0.0   

                               p1                        \
Link     IT0 0 H2 Electrolysis_nd IT0 0 H2 Fuel Cell_nd   
snapshot                                                  
0                       -0.000000                  -0.0   
1                       -0.000000                  -0.0   
2                       -0.000000                  -0.0   
3                       -0.000000                  -0.0   
4                     -100.000000                  -0.0   
5                       -0.000000                  -0.0   
6                       -0.000000                  -0.0   
7                       -0.000000                  -0.0   
8                       -0.000000                  -0.0   
9                      -32.253594                  -0.0   
10                    

In [ ]:
# Take the four series/single-col DF and give them clear names
p_nd  = nd.n.stores_t['p'].squeeze().rename('p_smspp')   # net power (nd)
p_net = network.stores_t['p'].squeeze().rename('p_pypsa')

e_nd  = nd.n.stores_t['e'].squeeze().rename('e_smspp')         # energy/SoC (nd)
e_net = network.stores_t['e'].squeeze().rename('e_pypsa')

# Optional sanity check: same index (snapshots)
assert p_nd.index.equals(p_net.index) and p_nd.index.equals(e_nd.index) and p_nd.index.equals(e_net.index), \
    "Indices must match"

# Simple side-by-side merge
pd.concat([p_nd, p_net, e_nd, e_net], axis=1)

# Now `merged` has columns: ['p_nd', 'p_net', 'e_nd', 'e_net']

,p_smspp,p_pypsa,e_smspp,e_pypsa
snapshot,,,,
0,0.000000,0.00,0.00,0.00
1,0.000000,0.00,0.00,0.00
2,0.000000,0.00,0.00,0.00
3,0.000000,0.00,0.00,0.00
4,-100.000000,0.00,57.73,0.00
5,0.000000,-57.73,57.73,57.73
6,0.000000,-57.73,57.73,115.46
7,0.000000,-57.73,57.73,173.19
8,0.000000,-57.73,57.73,230.92


In [ ]:
df1 = nd.n.generators_t['p']      # primo DataFrame
df2 = network.loads_t['p']   # secondo DataFrame
df1.join(df2, how='inner', lsuffix='_nd', rsuffix='_net')

,IT0 0 0 solar,slack_unit IT0 0,IT0 0
snapshot,,,
0,0.000000,1788.234222,1788.234222
1,0.000000,1771.016291,1771.016291
2,0.000000,1479.653136,1479.653136
3,0.000000,1545.240888,1545.240888
4,1454.054023,0.000000,1354.054023
5,1313.325885,0.000000,1313.325885
6,1215.013197,0.000000,1215.013197
7,2474.758636,0.000000,2474.758636
8,2706.948297,0.000000,2706.948297


In [ ]:
transformation.unitblocks['DCNetworkBlock_3']

{'enumerate': 'UnitBlock_3',
 'block': 'DCNetworkBlock_links',
 'name': 'IT0 0 H2 Electrolysis',
 'FlowValue': masked_array(data=[  0.        ,   0.        ,   0.        ,   0.        ,
                    100.        ,   0.        ,   0.        ,   0.        ,
                      0.        ,  32.25359432, 100.        , 100.        ,
                    100.        , 100.        , 100.        ,   0.        ,
                      0.        ,   0.        ,   0.        ,   0.        ,
                      0.        ,   0.        ,   0.        ,   0.        ],
              mask=False,
        fill_value=1e+20),
 'DualCost': masked_array(data=[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
                    -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
                    -0., -0.],
              mask=False,
        fill_value=1e+20),
 'DesignVariable': 100.0}

In [ ]:
transformation.unitblocks['IntermittentUnitBlock_0']

{'name': 'IT0 0 0 solar',
 'enumerate': 'UnitBlock_0',
 'block': 'IntermittentUnitBlock',
 'DesignVariable': array([19711.86918373]),
 'Extendable': True,
 'variables': {'Gamma': {'value': 0.0, 'type': 'float', 'size': ()},
  'Kappa': {'value': 1.0, 'type': 'float', 'size': ()},
  'MaxPower': {'value': array([[0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.213],
          [0.436],
          [0.593],
          [0.7  ],
          [0.764],
          [0.786],
          [0.772],
          [0.72 ],
          [0.627],
          [0.49 ],
          [0.288],
          [0.037],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ],
          [0.   ]]),
   'type': 'float',
   'size': ('TimeHorizon',)},
  'MinPower': {'value': array([[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          